In [7]:
# python $SUMO_HOME/tools/randomTrips.py -n config/osm.net.xml.gz -r routes.rou.xml -e 10000 -l --insertion-density=12

In [1]:
import os
import sys
import random
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import sumolib
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import sys
import io
from contextlib import redirect_stdout


if 'SUMO_HOME' in os.environ:
    print('SUMO_HOME found')
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import traci
from sumolib import checkBinary
# sumoBinary = checkBinary('sumo-gui')
sumoBinary = checkBinary('sumo')
roadNetwork = "./config/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", roadNetwork, "--start", "--quit-on-end"]
# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

SUMO_HOME found
Using device: cuda


In [2]:
def randomTrips(dur=10000, density=12):
    os.system("python $SUMO_HOME/tools/randomTrips.py -n config/osm.net.xml.gz -r config/osm.passenger.trips.xml -e " + str(dur) + " -l --insertion-density=" + str(density))

In [3]:
def shouldContinueSim():
    numVehicles = traci.simulation.getMinExpectedNumber()
    return True if numVehicles > 0 else False

In [4]:
def restart(dur, density):
    with io.StringIO() as buf, redirect_stdout(buf):
        try:
            traci.close()
        except:
            pass
        randomTrips(dur, density)
        traci.start(sumoCmd)
        step = 0
        for i in range(100):
            traci.simulationStep()
            step += 1

In [5]:
def zeros2end(x, length=None):
    mask = x == 0
    out = torch.stack(
        [torch.cat([x[_, :][~mask[_, :]], x[_, :][mask[_, :]]], dim=0) for _ in range(x.size()[0])]).to(device)
    if length is not None and length < out.size()[1]:
        out = out[:, :length]
    elif length is not None and length > out.size()[1]:
        out = torch.cat([out, torch.zeros(out.size()[0], length - out.size()[1]).to(device)], dim=1)
    return out.to(device)

In [6]:
def intervehicleConnectivity(threshold = None):
    xs = []
    ys = []
    for vehicle in traci.vehicle.getIDList():
        x, y = traci.vehicle.getPosition(vehicle)
        xs.append(x)
        ys.append(y)
    xs = torch.tensor(xs, dtype=torch.float32).to(device).view(-1,1)
    ys = torch.tensor(ys, dtype=torch.float32).to(device).view(-1,1)
    intervehicle_distances = torch.sqrt((xs - xs.t())**2 + (ys - ys.t())**2)
    if threshold is not None:
        # # make only the distances less than the threshold non-zero, the rest
        # intervehicle_distances = torch.where(intervehicle_distances < threshold, intervehicle_distances, torch.zeros_like(intervehicle_distances))
        # make the distances 1 if less than the threshold, 0 otherwise
        intervehicle_distances = torch.where(intervehicle_distances < threshold, torch.ones_like(intervehicle_distances), torch.zeros_like(intervehicle_distances))
    return intervehicle_distances, xs, ys

In [7]:
def getVehicleData(connectivity, xs, ys, length=500):
    masked_xs = torch.matmul(connectivity, torch.diag(xs.view(-1)))
    masked_ys = torch.matmul(connectivity, torch.diag(ys.view(-1)))
    vehicle_accelerations = torch.tensor([traci.vehicle.getAcceleration(vehicle) for vehicle in traci.vehicle.getIDList()], dtype=torch.float32).to(device).view(-1,1)
    vehicle_speeds = torch.tensor([traci.vehicle.getSpeed(vehicle) for vehicle in traci.vehicle.getIDList()], dtype=torch.float32).to(device).view(-1,1)
    vehicle_directions = torch.tensor([traci.vehicle.getAngle(vehicle) for vehicle in traci.vehicle.getIDList()], dtype=torch.float32).to(device).view(-1,1)
    masked_accelerations = torch.matmul(connectivity, torch.diag(vehicle_accelerations.view(-1)))
    masked_speeds = torch.matmul(connectivity, torch.diag(vehicle_speeds.view(-1)))
    masked_directions = torch.matmul(connectivity, torch.diag(vehicle_directions.view(-1)))
    vehicle_data = torch.cat([masked_xs.unsqueeze(2), masked_ys.unsqueeze(2), masked_accelerations.unsqueeze(2), masked_speeds.unsqueeze(2), masked_directions.unsqueeze(2)], dim=2)
    vehicle_data = vehicle_data.view(vehicle_data.size(0), -1)
    vehicle_data = zeros2end(vehicle_data, length)
    return vehicle_data

In [8]:
def getLabels(prev_connectiviy, prev_ids, length=500):
    # get all vehicle's current x, y positions
    curr_ids = traci.vehicle.getIDList()
    removed_ids = list(set(prev_ids) - set(curr_ids))
    xs = []
    ys = []
    for vehicle in prev_ids:
        if vehicle in removed_ids:
            xs.append(-1)
            ys.append(-1)
        else:
            x, y = traci.vehicle.getPosition(vehicle)
            xs.append(x)
            ys.append(y)
    xs = torch.tensor(xs, dtype=torch.float32).to(device).view(-1,1)
    ys = torch.tensor(ys, dtype=torch.float32).to(device).view(-1,1)
    # do pointwise multiplication of xs with prev_connectivity
    masked_xs = torch.mul(prev_connectiviy, xs).T
    masked_ys = torch.mul(prev_connectiviy, ys).T
    vehicle_labels = torch.cat([masked_xs.unsqueeze(2), masked_ys.unsqueeze(2)], dim=2).to(device)
    vehicle_labels = vehicle_labels.view(vehicle_labels.size(0), -1).to(device)
    vehicle_labels = zeros2end(vehicle_labels, length).to(device)
    return vehicle_labels

In [9]:
net = sumolib.net.readNet('./config/osm.net.xml.gz')

# Extract nodes and edges
nodes = net.getNodes()
edges = net.getEdges()

# Create a mapping from node IDs to matrix indices
node_ids = [node.getID() for node in nodes]
node_index = {node_id: idx for idx, node_id in enumerate(node_ids)}

# Initialize adjacency matrix
adj_matrix = np.zeros((len(nodes), len(nodes)))

# Populate adjacency matrix
for edge in edges:
    from_node = edge.getFromNode().getID()
    to_node = edge.getToNode().getID()
    from_idx = node_index[from_node]
    to_idx = node_index[to_node]
    adj_matrix[from_idx, to_idx] = 1  # or use edge attributes like edge.getLength()

# Optionally, extract edge features
edge_features = []
for edge in edges:
    # get number of lanes, speed limit, etc.
    num_lanes = edge.getLaneNumber()
    speed_limit = edge.getSpeed()
    length = edge.getLength()
    # get position of two nodes
    from_node = edge.getFromNode().getID()
    to_node = edge.getToNode().getID()
    from_idx = node_index[from_node]
    to_idx = node_index[to_node]
    edge_features.append((num_lanes, speed_limit, length, from_idx, to_idx))
# Optionally, extract node features
node_features = []
for node in nodes:
    node_pos = node.getCoord()
    node_type = 1 if node.getType() == "traffic_light" else 0
    node_features.append((node_pos[0], node_pos[1], node_type))

print("Adjacency Matrix:\n", adj_matrix.shape)
print("Edge Features:\n", len(edge_features))
print("Node Features:\n", len(node_features))
adj_matrix = torch.tensor(adj_matrix, dtype=torch.float32).to(device)
edge_features = torch.tensor(edge_features, dtype=torch.float32).to(device)
node_features = torch.tensor(node_features, dtype=torch.float32).to(device)

Adjacency Matrix:
 (354, 354)
Edge Features:
 740
Node Features:
 354


In [10]:
# Convert your data to PyTorch Geometric format
def prepare_data(adj_matrix, node_features, edge_features=None):
    edge_index = torch.tensor(adj_matrix.nonzero(), dtype=torch.long).T
    x = torch.tensor(node_features, dtype=torch.float)
    
    # If you have edge features, you can add them as edge attributes
    if edge_features is not None:
        edge_attr = torch.tensor(edge_features, dtype=torch.float)
        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    else:
        data = Data(x=x, edge_index=edge_index)
    
    return data

## Check if there are parallel edges (multigrpah)

In [11]:
net = sumolib.net.readNet('./config/osm.net.xml.gz')

# Extract nodes and edges
nodes = net.getNodes()
edges = net.getEdges()

edges_from_to = {}
for edge in edges:
    from_node = edge.getFromNode().getID()
    to_node = edge.getToNode().getID()
    if (from_node, to_node) not in edges_from_to:
        edges_from_to[(from_node, to_node)] = [edge]
    else:
        edges_from_to[(from_node, to_node)].append(edge)

for from_node, to_node in edges_from_to.keys():
    if len(edges_from_to[(from_node, to_node)]) > 1:
        print(from_node, to_node)
        for edge in edges_from_to[(from_node, to_node)]:
            print(edge.getID())
            num_lanes = edge.getLaneNumber()
            speed_limit = edge.getSpeed()
            length = edge.getLength()

In [12]:
prepare_data(adj_matrix, node_features, edge_features).to(device)

/tmp/ipykernel_4268/1013685065.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(adj_matrix.nonzero(), dtype=torch.long).T
/tmp/ipykernel_4268/1013685065.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(node_features, dtype=torch.float)
/tmp/ipykernel_4268/1013685065.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(edge_features, dtype=torch.float)


Data(x=[354, 3], edge_index=[2, 740], edge_attr=[740, 5])

## Define the model (with GNN)

In [13]:
# Define the GCN model
class TrafficGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, out_dim=256):
        super(TrafficGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.fc = nn.Linear(out_channels * 354, out_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x).view(-1)
        x = self.fc(x)
       
        return F.relu(x)

class Encoder(torch.nn.Module):
    def __init__(self, gnn_out_channels, neigh_vec_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.fc1 = torch.nn.Linear(gnn_out_channels + neigh_vec_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = torch.nn.Linear(hidden_dim, hidden_dim)

    def forward(self, gnn_output, neigh_vehicles_vector):
        combined = torch.cat([gnn_output, neigh_vehicles_vector])
        x = F.relu(self.fc1(combined))
        encoded = F.relu(self.fc2(x))
        encoded = F.relu(self.fc3(encoded))
        return encoded
    
class Decoder(torch.nn.Module):
    def __init__(self, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim)
        self.fc3 = torch.nn.Linear(output_dim, output_dim)

    def forward(self, encoded):
        x = F.leaky_relu(self.fc1(encoded))
        output = F.leaky_relu(self.fc2(x))
        output = F.leaky_relu(self.fc3(output))

        return output

class TrafficPredictionModel(torch.nn.Module):
    def __init__(self, in_channels, gnn_hidden, gnn_out, gnn_out_dim, neigh_vec_dim, encoder_hidden, output_dim):
        super(TrafficPredictionModel, self).__init__()
        self.gnn = TrafficGCN(in_channels, gnn_hidden, gnn_out, out_dim=gnn_out_dim)
        self.encoder = Encoder(gnn_out_dim, neigh_vec_dim, encoder_hidden)
        self.decoder = Decoder(encoder_hidden, output_dim)

    def forward(self, data, neigh_vehicles_vector):
        gnn_output = self.gnn(data)
        encoded = self.encoder(gnn_output, neigh_vehicles_vector)
        prediction = self.decoder(encoded)
        return prediction

In [30]:
try:
    traci.close()
except:
    pass
import torch.optim as optim
# Instantiate the model
dur = 10000
density = 36
vehicle_data_length = 100
dsrc_range = 500
randomTrips(dur, density)
traci.start(sumoCmd)
step = 0
for i in range(100):
    traci.simulationStep()
    step += 1
connectivity, xs, ys = intervehicleConnectivity(dsrc_range)
neigh_vehicles_vector = getVehicleData(connectivity, xs, ys, vehicle_data_length * 5).to(device)
old_ids = traci.vehicle.getIDList()


interval_steps = 3
for i in range(interval_steps):
    traci.simulationStep()


target_positions = getLabels(connectivity, old_ids, vehicle_data_length * 2).to(device)
model = TrafficPredictionModel(in_channels=node_features.shape[1], gnn_hidden=8, gnn_out=2, gnn_out_dim=256,
                               neigh_vec_dim=neigh_vehicles_vector.shape[1], encoder_hidden=256, output_dim=target_positions.shape[1]).to(device)
print(model)
# Define loss and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Example data (placeholders)
data = prepare_data(adj_matrix, node_features, edge_features).to(device)

# Training loop
for epoch in range(500):
    for i in range(len(neigh_vehicles_vector)):
        each = neigh_vehicles_vector[i]
        model.train()
        optimizer.zero_grad()
        prediction = model(data, each)
        loss = criterion(prediction, target_positions[i])
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Loss: {loss.item()}')
    if loss.item() < 50:
        break

Simulation ended at time: 105.00
Reason: TraCI requested termination.
Performance: 
 Duration: 52.73s
 TraCI-Duration: 0.02s
 Real time factor: 1.99113
 UPS: 51.219327
Vehicles: 
 Inserted: 52 (Loaded: 99)
 Running: 49
 Waiting: 0
Statistics (avg of 3):
 RouteLength: 493.52
 Speed: 6.67
 Duration: 72.33
 WaitingTime: 1.00
 TimeLoss: 13.18
 DepartDelay: 0.37

Success.
 Retrying in 1 seconds
***Starting server on port 60313 ***
Loading net-file from './config/osm.net.xml.gz' ... done (62ms).
Loading additional-files from './config/osm.poly.xml.gz' ... done (160ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
TrafficPredictionModel(
  (gnn): TrafficGCN(
    (conv1): GCNConv(3, 8)
    (conv2): GCNConv(8, 2)
    (fc): Linear(in_features=708, out_features=256, bias=True)
  )
  (encoder): Encoder(
    (fc1): Linear(in_features=756, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=256, bias=True)
    (fc3): Linear(in_features=256, out_features=2

/tmp/ipykernel_5631/1013685065.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(adj_matrix.nonzero(), dtype=torch.long).T
/tmp/ipykernel_5631/1013685065.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(node_features, dtype=torch.float)
/tmp/ipykernel_5631/1013685065.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(edge_features, dtype=torch.float)


Epoch 1, Loss: 40041.83203125
Epoch 2, Loss: 39569.1015625
Epoch 3, Loss: 40000.8125
Epoch 4, Loss: 39453.796875
Epoch 5, Loss: 39559.61328125
Epoch 6, Loss: 39334.36328125
Epoch 7, Loss: 38756.48828125
Epoch 8, Loss: 36349.51953125
Epoch 9, Loss: 36912.0234375
Epoch 10, Loss: 38067.0078125
Epoch 11, Loss: 37726.37109375


KeyboardInterrupt: 

## More serious training

In [66]:
try:
    traci.close()
except:
    pass
import torch.optim as optim
# Instantiate the model
dur = 5000
density = 36
vehicle_data_length = 500
dsrc_range = 500
interval_steps = 5
randomTrips(dur, density)
traci.start(sumoCmd)
step = 0
for i in range(100):
    traci.simulationStep()
    step += 1

data = prepare_data(adj_matrix, node_features, edge_features).to(device)

connectivity, xs, ys = intervehicleConnectivity(500)
neigh_vehicles_vector = getVehicleData(connectivity, xs, ys, vehicle_data_length).to(device)
old_ids = traci.vehicle.getIDList()

for i in range(interval_steps):
    traci.simulationStep()
    step += 1

target_positions = getLabels(connectivity, old_ids, vehicle_data_length).to(device)

model = TrafficPredictionModel(in_channels=node_features.shape[1], gnn_hidden=8, gnn_out=2, gnn_out_dim=256,
                            neigh_vec_dim=neigh_vehicles_vector.shape[1], encoder_hidden=256, output_dim=target_positions.shape[1]).to(device)
# Define loss and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

print("start training")
while step < dur and shouldContinueSim():
    print(target_positions.shape)
    print(neigh_vehicles_vector.shape)
    print(len(traci.vehicle.getIDList()))
    # Training loop
    for i in range(len(neigh_vehicles_vector)):
        each = neigh_vehicles_vector[i]
        model.train()
        optimizer.zero_grad()
        prediction = model(data, each)
        loss = criterion(prediction, target_positions[i])
        loss.backward()
        optimizer.step()

    connectivity, xs, ys = intervehicleConnectivity(500)
    neigh_vehicles_vector = getVehicleData(connectivity, xs, ys, vehicle_data_length * 5).to(device)
    old_ids = traci.vehicle.getIDList()

    for i in range(interval_steps):
        traci.simulationStep()
        step += 1
    print(f'step {step}, Loss: {loss.item()}')

    target_positions = getLabels(connectivity, old_ids, vehicle_data_length * 2).to(device)

Simulation ended at time: 105.00
Reason: TraCI requested termination.
Performance: 
 Duration: 34.06s
 TraCI-Duration: 0.02s
 Real time factor: 3.08252
 UPS: 79.294249
Vehicles: 
 Inserted: 52 (Loaded: 99)
 Running: 49
 Waiting: 0
Statistics (avg of 3):
 RouteLength: 493.52
 Speed: 6.67
 Duration: 72.33
 WaitingTime: 1.00
 TimeLoss: 13.18
 DepartDelay: 0.37

Success.
 Retrying in 1 seconds
***Starting server on port 33527 ***
Loading net-file from './config/osm.net.xml.gz' ... done (64ms).
Loading additional-files from './config/osm.poly.xml.gz' ... done (161ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
start training
torch.Size([48, 500])
torch.Size([48, 500])
49
step 110, Loss: 16027.94921875
torch.Size([49, 1000])
torch.Size([49, 2500])
49


/tmp/ipykernel_5631/1013685065.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(adj_matrix.nonzero(), dtype=torch.long).T
/tmp/ipykernel_5631/1013685065.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(node_features, dtype=torch.float)
/tmp/ipykernel_5631/1013685065.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(edge_features, dtype=torch.float)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x2756 and 756x256)

In [248]:
def check(i):
    model.eval()
    with torch.no_grad():
        prediction = model(data, neigh_vehicles_vector[i])
        print(prediction)
        print(target_positions[i])
check(15)

tensor([ 1.4192e+03,  4.7533e+02,  1.1898e+03,  1.0082e+03,  1.4986e+03,
         5.6443e+02,  1.7111e+03,  9.0397e+02,  1.8483e+03,  8.4258e+02,
         1.7902e+03,  5.9587e+02,  2.0070e+01, -1.9692e-01,  1.9048e+03,
         6.4869e+02,  1.8423e+03,  8.7527e+02,  1.3950e+03,  5.8599e+02,
         1.5090e+03,  5.3803e+02,  1.4525e+03,  7.8797e+02,  1.7451e+03,
         9.1228e+02, -1.7069e+00, -5.0556e+00, -9.7885e+00, -6.1189e+00,
        -7.1793e+00, -1.6085e+01, -7.2459e+00, -1.2310e-01, -7.6340e+00,
        -2.1725e+00, -1.0961e+01, -4.6848e+00, -1.4038e+01, -6.8130e+00,
        -8.9812e+00, -1.9745e+00, -6.3635e+00, -1.7564e+00, -2.4400e+00,
        -3.6481e+00, -2.0685e+00, -6.1219e-01, -2.6925e+00, -4.0849e+00,
        -1.9142e+00, -1.8652e+00, -1.6841e+00, -2.0559e+00, -2.1094e+00,
        -3.2438e+00, -2.1866e+00, -2.0920e+00, -3.4903e+00, -6.1892e+00,
        -3.5573e+00, -8.9187e+00, -5.6867e+00, -1.1584e+01, -1.2474e+01,
        -7.8456e+00, -3.4483e-01, -7.8448e+00, -9.3

## Define the model (without GNN)

In [12]:
class TrafficPredictionModel(torch.nn.Module):
    def __init__(self, neigh_vec_dim, hidden, output_dim):
        super(TrafficPredictionModel, self).__init__()
        self.fc1 = torch.nn.Linear(neigh_vec_dim, hidden)
        self.fc2 = torch.nn.Linear(hidden, output_dim)

    def forward(self, neigh_vehicles_vector):
        x = F.relu(self.fc1(neigh_vehicles_vector))
        x = F.relu(self.fc2(x))
        return x

## Train the NON-GNN network

In [24]:
# Generate synthetic data
def generate_data(vehicle_data_length, interval_steps=5, dsrc_range=500, dur=5000, density=6):
    restart(dur, density)
    step = 0
    inputs = []
    targets = []
    while step < dur and shouldContinueSim():
        connectivity, xs, ys = intervehicleConnectivity(dsrc_range)
        neigh_vehicles_vector = getVehicleData(connectivity, xs, ys, vehicle_data_length * 5).to(device)
        old_ids = traci.vehicle.getIDList()

        for i in range(interval_steps):
            traci.simulationStep()
            step += 1

        target_positions = getLabels(connectivity, old_ids, vehicle_data_length * 2).to(device)
        inputs.append(neigh_vehicles_vector[0])
        targets.append(target_positions[0])
    inputs = torch.stack(inputs)
    targets = torch.stack(targets)
    dataset = TensorDataset(inputs, targets)  # Create a TensorDataset
    return DataLoader(dataset, batch_size=100, shuffle=True)
        

# Training the model
def train(model, criterion, optimizer, num_epochs, vehicle_data_length, interval_steps=5, dsrc_range=500, dur=5000, density=6):
    model.train()
    for epoch in range(num_epochs):
        # Get new data each epoch
        dataloader = generate_data(vehicle_data_length, interval_steps, dsrc_range, dur, density)
        total_loss = 0
        for inputs, targets in dataloader:
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, targets)

            # Backward pass
            loss.backward()

            # Update parameters
            optimizer.step()

            total_loss += loss.item()

        if (epoch+1) % 10 == 0:
            avg_loss = total_loss / len(dataloader)
            print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')


In [25]:
# Instantiate the model
dur = 5000
density = 6
vehicle_data_length = 20
dsrc_range = 5000
interval_steps = 5
restart(dur, density)

connectivity, xs, ys = intervehicleConnectivity(dsrc_range)
neigh_vehicles_vector = getVehicleData(connectivity, xs, ys, vehicle_data_length * 5).to(device)
old_ids = traci.vehicle.getIDList()
step = 0
for i in range(interval_steps):
    traci.simulationStep()
    step += 1

target_positions = getLabels(connectivity, old_ids, vehicle_data_length * 2).to(device)
model = TrafficPredictionModel(neigh_vec_dim=neigh_vehicles_vector.shape[1], hidden=128, output_dim=target_positions.shape[1]).to(device)
# Define loss and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
train(model, criterion, optimizer, num_epochs, vehicle_data_length, interval_steps, dsrc_range, dur, density)

Simulation ended at time: 5100.00
Reason: TraCI requested termination.
Performance: 
 Duration: 1308.39s
 TraCI-Duration: 3.99s
 Real time factor: 3.89793
 UPS: 59.103308
Vehicles: 
 Inserted: 380
 Running: 9
 Waiting: 0
Statistics (avg of 371):
 RouteLength: 1584.57
 Speed: 7.75
 Duration: 204.35
 WaitingTime: 11.12
 TimeLoss: 36.81
 DepartDelay: 0.53

Success.
***Starting server on port 45023 ***
Loading net-file from './config/osm.net.xml.gz' ... done (67ms).
Loading additional-files from './config/osm.poly.xml.gz' ... done (161ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
Simulation ended at time: 105.00
Reason: TraCI requested termination.
Performance: 
 Duration: 0.40s
 TraCI-Duration: 0.02s
 Real time factor: 265.152
 UPS: 1401.515152
Vehicles: 
 Inserted: 10 (Loaded: 18)
 Running: 10
 Waiting: 0
Statistics (avg of 0):
 RouteLength: 0.00
 Speed: 0.00
 Duration: 0.00
 WaitingTime: 0.00
 TimeLoss: 0.00
 DepartDelay: 0.00

Success.
***Starting server on port

KeyboardInterrupt: 

In [ ]:
def check(i):
    model.eval()
    with torch.no_grad():
        prediction = model(neigh_vehicles_vector[i])
        print(prediction)
        print(target_positions[i])
check(15)

tensor([ 8.1872e+02,  4.9300e+02,  6.7327e+02,  4.0129e+02,  1.0266e+03,
         6.1192e+02,  1.0235e+03,  5.7670e+02,  1.2238e+03,  5.9713e+02,
         1.0596e+03,  5.5669e+02,  9.8745e+02,  5.1876e+02,  8.3305e+02,
         4.0075e+02,  8.1763e+02,  3.7739e+02,  1.0219e+03,  4.5800e+02,
         8.1804e+02,  3.6240e+02,  8.8421e+02,  4.0961e+02,  7.6787e+02,
         3.7227e+02,  7.9919e+02,  3.0671e+02,  1.0303e+03,  4.2392e+02,
         9.4643e+02,  3.8858e+02,  8.1903e+02,  4.5717e+02,  8.7270e+02,
         3.9464e+02,  9.0492e+02,  3.7391e+02,  8.3559e+02,  4.0438e+02,
         9.1100e+02,  3.9342e+02,  9.4737e+02,  4.2318e+02,  8.1770e+02,
         3.8515e+02,  7.9423e+02,  4.0380e+02,  8.4357e+02,  3.9070e+02,
         8.4944e+02,  3.9370e+02,  8.7002e+02,  4.7568e+02,  8.0487e+02,
         4.4310e+02,  8.8095e+02,  4.0253e+02,  7.6027e+02,  4.2968e+02,
         8.9714e+02,  3.7803e+02,  8.3495e+02,  3.8067e+02,  9.1753e+02,
         4.5364e+02,  8.0439e+02,  3.5815e+02,  9.1

In [5]:
class TrafficPredictionModel(torch.nn.Module):
    def __init__(self, neigh_vec_dim, hidden, output_dim):
        super(TrafficPredictionModel, self).__init__()
        self.fc1 = torch.nn.Linear(neigh_vec_dim, hidden)
        self.fc2 = torch.nn.Linear(hidden, hidden)
        self.fc3 = torch.nn.Linear(hidden, hidden)
        self.fc4 = torch.nn.Linear(hidden, output_dim)

    def forward(self, neigh_vehicles_vector):
        x = F.relu(self.fc1(neigh_vehicles_vector))
        # x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return x

In [6]:
i = 100
model = TrafficPredictionModel(2 * i, 1024, i).to(device)

# Loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss
optimizer = optim.SGD(model.parameters(), lr=0.0005)

# Generate synthetic data
def generate_data(num_samples, i=1):
    inputs = torch.rand(num_samples*i, 2) * 1000
    targets = inputs.sum(dim=1, keepdim=True)  # Sums of each pair
    inputs = inputs.view(num_samples, 2 * i).to(device)
    targets = targets.view(num_samples, i).to(device)
    dataset = TensorDataset(inputs, targets)  # Create a TensorDataset
    return DataLoader(dataset, batch_size=100, shuffle=True)  # Create a DataLoader for batching

# Training the model
def train(model, criterion, optimizer, num_epochs, num_samples, i):
    model.train()
    for epoch in range(num_epochs):
        dataloader = generate_data(num_samples, i)  # Get DataLoader
        total_loss = 0
        for inputs, targets in dataloader:
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, targets)

            # Backward pass
            loss.backward()

            # Update parameters
            optimizer.step()

            total_loss += loss.item()

        if (epoch+1) % 10 == 0:
            avg_loss = total_loss / len(dataloader)
            print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

# Parameters
num_epochs = 1000
num_samples = 1000

# Train the model
train(model, criterion, optimizer, num_epochs, num_samples, i)

Epoch [10/1000], Average Loss: 1172226.4500
Epoch [20/1000], Average Loss: 1162493.5125
Epoch [30/1000], Average Loss: 1170547.1625
Epoch [40/1000], Average Loss: 1168169.8250
Epoch [50/1000], Average Loss: 1167527.1500
Epoch [60/1000], Average Loss: 1163255.2000
Epoch [70/1000], Average Loss: 1165141.4000
Epoch [80/1000], Average Loss: 1166388.5500
Epoch [90/1000], Average Loss: 1166059.7125
Epoch [100/1000], Average Loss: 1159930.8375
Epoch [110/1000], Average Loss: 1165152.8500
Epoch [120/1000], Average Loss: 1168146.0250
Epoch [130/1000], Average Loss: 1163268.2875
Epoch [140/1000], Average Loss: 1166722.4625
Epoch [150/1000], Average Loss: 1163611.3375
Epoch [160/1000], Average Loss: 1166893.2625
Epoch [170/1000], Average Loss: 1168797.4875
Epoch [180/1000], Average Loss: 1164545.7125
Epoch [190/1000], Average Loss: 1162751.3625
Epoch [200/1000], Average Loss: 1166766.8500
Epoch [210/1000], Average Loss: 1166899.3500
Epoch [220/1000], Average Loss: 1165801.7875
Epoch [230/1000], A

In [64]:
# traci.start(sumoCmd)
# step = 0
# def rerouteVehicles():
#     global vehicles
#     routes_status = {}
#     time = traci.simulation.getTime()
#     for vehicle in traci.vehicle.getIDList():
#         if vehicle not in vehicles or vehicles[vehicle]-time > 50:
#             vehicles[vehicle] = time
#         else:
#             continue
#         if random.random() > 0.2:
#             continue
#         for link in traci.vehicle.getNextLinks(vehicle):
#             edge = traci.lane.getEdgeID(link[0])
#             if link[0] not in routes_status:
#                     ratio = traci.lane.getLastStepMeanSpeed(link[0])/ traci.lane.getMaxSpeed(link[0])
#                     routes_status[link[0]] = ratio
#             else:
#                     ratio = routes_status[link[0]]
#             if ratio < 0.3:
#                 traci.vehicle.setAdaptedTraveltime(vehicle, edge, float('inf'), endTime=time+3)
#         traci.vehicle.rerouteTraveltime(vehicle)
# while step < 10000 and shouldContinueSim():
#     traci.simulationStep()
#     vehicles = {}
#     if random.random() < 0.1:
#         rerouteVehicles()
#     step += 1
# traci.close()

In [ ]:
# # Instantiate the model
# dur = 10000
# density = 36
# vehicle_data_length = 500
# randomTrips(dur, density)
# traci.start(sumoCmd)
# step = 0
# for i in range(100):
#     traci.simulationStep()
#     step += 1
# threshold = 400
# connectivity, xs, ys = intervehicleConnectivity(threshold)
# vehicle_data_length = 500
# neigh_vehicles_vector = getVehicleData(connectivity, xs, ys, vehicle_data_length)
# labels = getLabels(connectivity, traci.vehicle.getIDList(), vehicle_data_length)[0]
# print(labels)
# ids = traci.vehicle.getIDList()
# c = 0
# for i in range(len(connectivity[0])):
#     if connectivity[0][i] == 1:
#         assert labels[c] == traci.vehicle.getPosition(ids[i])[0]
#         assert labels[c+1] == traci.vehicle.getPosition(ids[i])[1]
#         c += 2
# assert labels[c] == 0